In [3]:
from huggingface_hub import login
import torch
import os
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from sentence_transformers import CrossEncoder
from dotenv import load_dotenv


d:\Tugas\SMT 7\Skripsi\Scripts\DrugsBot\drugbot-backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
token = os.getenv("HF_TOKEN")

login(token=token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
embedding_id = "distiluse-base-multilingual-cased-v2"

embeddings = HuggingFaceEmbeddings(model_name=embedding_id, cache_folder="./model_cache")

In [5]:
reranker_id = "cross-encoder/ms-marco-MiniLM-L-6-v2"

reranker = CrossEncoder(model_name=reranker_id, cache_dir="./model_cache")

In [6]:
reranker_2 = HuggingFaceCrossEncoder(model_name=reranker_id, model_kwargs={'cache_dir' : "./model_cache"})

In [7]:
import torch
from transformers import pipeline

In [8]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    model_kwargs={'cache_dir' : './model_cache'},
    max_new_tokens=256
)

ValueError: Could not load model meta-llama/Llama-3.2-1B-Instruct with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "d:\Tugas\SMT 7\Skripsi\Scripts\DrugsBot\drugbot-backend\.venv\Lib\site-packages\transformers\pipelines\base.py", line 291, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Tugas\SMT 7\Skripsi\Scripts\DrugsBot\drugbot-backend\.venv\Lib\site-packages\transformers\models\auto\auto_factory.py", line 571, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Tugas\SMT 7\Skripsi\Scripts\DrugsBot\drugbot-backend\.venv\Lib\site-packages\transformers\modeling_utils.py",

In [6]:
llm = HuggingFacePipeline(pipeline=pipe)

In [7]:
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

question = "What is electroencephalography?"

response = chain.invoke({"question": question})

print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: What is electroencephalography?

Answer: Let's think step by step. Electroencephalography (EEG) is a diagnostic tool that records the electrical activity of the brain. This is done using electrodes placed on the scalp to capture the brain's electrical signals. These signals are typically measured in terms of their frequency, amplitude, and synchronization, which can be used to diagnose and monitor various neurological conditions.

Step 1: Identify the main purpose of EEG
The main purpose of EEG is to record the electrical activity of the brain.

Step 2: Consider the types of signals recorded by EEG
EEG records various types of signals, including:
- Brain waves: delta, theta, alpha, beta, and theta-4 wave activity.
- Cortical activity: local field potentials and synchronized neural activity.
- Transient activity: spike and wave discharges.

Step 3: Analyze the applications of EEG
EEG has a wide range of applications, including:
- Diagnosing and monitoring neurological conditio

In [1]:
from langchain_tavily import TavilySearch

In [4]:
tavily_key = os.getenv("TAVILY_API_KEY")

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = tavily_key

In [5]:
tool = TavilySearch(
    max_results=5,
    topic="general",
)

tool.invoke({"query": "Obat bpom untuk diabetes"})

{'query': 'Obat bpom untuk diabetes',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'Ini 5 Rekomendasi Obat Diabetes untuk Menurunkan Gula Darah ... - Halodoc',
   'url': 'https://www.halodoc.com/artikel/ini-5-rekomendasi-obat-diabetes-untuk-menurunkan-gula-darah-tinggi',
   'content': 'No registrasi BPOM: DKI2164605043A1. Kisaran harga: Rp2.671.100 per pen. ... Jenis Obat Diabetes untuk Menurunkan Gula Darah Tinggi. Penting untuk diingat, penggunaan obat-obatan berikut harus sesuai dengan resep dokter. Tujuannya agar dosisnya tepat dan sesuai dengan kondisi tubuh.',
   'score': 0.5726516,
   'raw_content': None},
  {'title': '9 Jenis Obat Diabetes yang Umum Diresepkan Dokter - Hello Sehat',
   'url': 'https://hellosehat.com/diabetes/obat-diabetes/',
   'content': 'Secara umum, golongan obat diabetes memiliki cara kerja dan efek samping yang berbeda. Namun, fungsinya tetap sama, yaitu membantu mengendalikan kadar gula darah sekaligus menekan risik